In [1]:
import sys

sys.path.append("../src")
from linearboost.linear_boost import LinearBoostClassifier

In [2]:
pip install xgboost lightgbm catboost

Note: you may need to restart the kernel to use updated packages.


In [20]:
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder

# The Huberman's Survival's id on UCI Machine Learning Repository
dataset_id = 52

dataset = fetch_ucirepo(id=dataset_id)

# data (as pandas dataframes)
X = dataset.data.features.copy()
y = dataset.data.targets

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y.values.ravel())
import numpy as np

y = np.where(np.isin(y, [2, 3, 4, 5]), 1, y)

In [12]:
y

array([1, 1, 1, ..., 0, 0, 0], shape=(4601,))

In [21]:
# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    X[col] = X[col].astype("category")

# Handle missing values
# Fill numeric columns with median
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
for col in numeric_cols:
    X[col] = X[col].fillna(X[col].median())

# Fill categorical columns with mode
for col in categorical_cols:
    X[col] = X[col].fillna(X[col].mode()[0])

In [22]:
import warnings

warnings.filterwarnings("ignore", message=".*ignore_implicit_zeros.*")
warnings.filterwarnings("ignore", message=".*n_quantiles.*")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

**LinearBoost results:**

In [27]:
import optuna
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline

# X, y assumed pre-defined (raw, with object/category cols intact)


def custom_loss(y_true, y_pred, weights):
    return np.mean(weights * (y_true - y_pred) ** 2)


def _make_preprocessor() -> ColumnTransformer:
    """
    Creates a preprocessor that only one-hot encodes categorical features
    and passes numerical features through without scaling.
    """
    return ColumnTransformer(
        transformers=[
            # The "cat" transformer applies OneHotEncoder to columns of type object or category.
            (
                "cat",
                OneHotEncoder(handle_unknown="ignore"),
                make_column_selector(dtype_include=["object", "category"]),
            ),
        ],
        # 'remainder="passthrough"' ensures that all other columns (i.e., numerical ones) are kept.
        remainder="passthrough",
        n_jobs=None,
    )


def objective(trial):
    """
    Optuna objective function for hyperparameter tuning.
    """
    # Define the search space for the classifier's parameters.
    # The "scaler" parameter has been removed.
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
        "algorithm": trial.suggest_categorical("algorithm", ["SAMME", "SAMME.R"]),
        "scaler": trial.suggest_categorical(
            "scaler", ["minmax", "robust", "quantile-uniform", "quantile-normal"]
        ),
        "kernel": trial.suggest_categorical(
            "kernel", ["linear", "rbf", "poly", "sigmoid"]
        ),
    }
    # Conditionally add parameters based on the chosen kernel.
    if params["kernel"] != "linear":
        params["gamma"] = trial.suggest_float("gamma", 1e-3, 10.0, log=True)
    if params["kernel"] == "poly":
        params["degree"] = trial.suggest_int("degree", 2, 5)
    if params["kernel"] in ["poly", "sigmoid"]:
        params["coef0"] = trial.suggest_float("coef0", 0.0, 1.0)

    # Build a leakage-free pipeline for the trial.
    # The preprocessor no longer requires a scaler argument.
    pre = _make_preprocessor()

    # All items in `params` are now intended for the classifier.
    clf = LinearBoostClassifier(**params)

    pipe = Pipeline(steps=[("preprocess", pre), ("model", clf)])

    try:
        # Perform stratified 10-fold cross-validation.
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = cross_val_score(pipe, X, y, scoring="f1_weighted", cv=cv)
        # Return the mean F1 score, or negative infinity if scores contain NaN.
        return -np.inf if np.isnan(scores).any() else scores.mean()
    except Exception as e:
        # Prune trial if an exception occurs (e.g., invalid parameter combination).
        print(f"Trial failed with exception: {e}")
        return -np.inf


# --- Optuna Study Execution ---
# Create a study object and specify the direction as "maximize" for F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# --- Print Best Results ---
print("\nBest trial:")
best_trial = study.best_trial
print(f"  Value (F1 Score): {best_trial.value:.4f}")
print("  Parameters: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-08-19 22:02:02,037] A new study created in memory with name: no-name-82933463-2c1e-432a-893e-8dccaf42a971


[I 2025-08-19 22:02:02,719] Trial 0 finished with value: 0.6404040390084329 and parameters: {'n_estimators': 98, 'learning_rate': 0.025040969870000332, 'algorithm': 'SAMME', 'scaler': 'quantile-normal', 'kernel': 'poly', 'gamma': 0.0036506834438673618, 'degree': 5, 'coef0': 0.3301750124911056}. Best is trial 0 with value: 0.6404040390084329.
[I 2025-08-19 22:02:03,124] Trial 1 finished with value: 0.5345075877473346 and parameters: {'n_estimators': 223, 'learning_rate': 0.7297126647372456, 'algorithm': 'SAMME.R', 'scaler': 'quantile-normal', 'kernel': 'poly', 'gamma': 0.8114762985242304, 'degree': 3, 'coef0': 0.427337620015356}. Best is trial 0 with value: 0.6404040390084329.
[I 2025-08-19 22:02:03,558] Trial 2 finished with value: 0.8435145697001879 and parameters: {'n_estimators': 400, 'learning_rate': 0.11869138844012254, 'algorithm': 'SAMME', 'scaler': 'quantile-normal', 'kernel': 'rbf', 'gamma': 0.005788104394731735}. Best is trial 2 with value: 0.8435145697001879.
[I 2025-08-19 2

Trial failed with exception: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/pipeline.py", line 662, in fit
    s

[I 2025-08-19 22:02:39,055] Trial 57 finished with value: 0.8717863235369474 and parameters: {'n_estimators': 257, 'learning_rate': 0.5013464948241906, 'algorithm': 'SAMME', 'scaler': 'quantile-normal', 'kernel': 'rbf', 'gamma': 0.4424401922925503}. Best is trial 53 with value: 0.9543374173629988.
[I 2025-08-19 22:02:39,588] Trial 58 finished with value: 0.6882396202694265 and parameters: {'n_estimators': 293, 'learning_rate': 0.31142751907125016, 'algorithm': 'SAMME', 'scaler': 'robust', 'kernel': 'poly', 'gamma': 3.613839721994584, 'degree': 4, 'coef0': 0.32280671741188566}. Best is trial 53 with value: 0.9543374173629988.
[I 2025-08-19 22:02:39,778] Trial 59 finished with value: 0.8819539998922167 and parameters: {'n_estimators': 233, 'learning_rate': 0.6097668080397359, 'algorithm': 'SAMME', 'scaler': 'minmax', 'kernel': 'rbf', 'gamma': 0.08883583450139441}. Best is trial 53 with value: 0.9543374173629988.
[I 2025-08-19 22:02:40,441] Trial 60 finished with value: 0.9281749115310631

Trial failed with exception: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/pipeline.py", line 662, in fit
    s

[I 2025-08-19 22:03:09,683] Trial 110 finished with value: 0.9354204269132287 and parameters: {'n_estimators': 223, 'learning_rate': 0.11525970851531234, 'algorithm': 'SAMME', 'scaler': 'minmax', 'kernel': 'rbf', 'gamma': 4.42390474054593}. Best is trial 86 with value: 0.9573878203541313.
[I 2025-08-19 22:03:10,337] Trial 111 finished with value: 0.9432228758248777 and parameters: {'n_estimators': 184, 'learning_rate': 0.2319177796947066, 'algorithm': 'SAMME', 'scaler': 'minmax', 'kernel': 'rbf', 'gamma': 1.3285933571091035}. Best is trial 86 with value: 0.9573878203541313.
[I 2025-08-19 22:03:10,826] Trial 112 finished with value: 0.9547394467306798 and parameters: {'n_estimators': 172, 'learning_rate': 0.45079109156398994, 'algorithm': 'SAMME', 'scaler': 'minmax', 'kernel': 'rbf', 'gamma': 2.2893364126109446}. Best is trial 86 with value: 0.9573878203541313.
[I 2025-08-19 22:03:11,421] Trial 113 finished with value: 0.9377302419765551 and parameters: {'n_estimators': 235, 'learning_r


Best trial:
  Value (F1 Score): 0.9574
  Parameters: 
    n_estimators: 193
    learning_rate: 0.26419239924556914
    algorithm: SAMME
    scaler: minmax
    kernel: rbf
    gamma: 2.768323929854739


**XGBoost results:**

In [ ]:
import xgboost as xgb


def objective(trial):
    params = {
        "objective": "binary:logistic",
        "use_label_encoder": False,
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.7),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "enable_categorical": True,
        "eval_metric": "logloss",
    }

    model = xgb.XGBClassifier(**params)

    scores = cross_val_score(
        estimator=model,
        X=X,
        y=y,
        scoring="f1_weighted",
        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
        n_jobs=-1,
    )

    return scores.mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

best_trial = study.best_trial

print("Best trial:")
print(f"F1 Score: {best_trial.value:.6f}")
print("Parameters:")
for k, v in best_trial.params.items():
    print(f"{k}: {v}")

[I 2025-08-18 23:04:37,538] A new study created in memory with name: no-name-a6ac08d4-0166-4e55-82db-e34c20f65e38
[I 2025-08-18 23:04:38,723] Trial 0 finished with value: 0.9636743691091517 and parameters: {'n_estimators': 793, 'max_depth': 16, 'learning_rate': 0.08966826500844771, 'gamma': 1.5332174950801975e-05, 'min_child_weight': 7, 'subsample': 0.8206767596857434, 'colsample_bytree': 0.5726809729609164, 'reg_alpha': 0.3000662922036043, 'reg_lambda': 6.694076855447791e-06}. Best is trial 0 with value: 0.9636743691091517.
[I 2025-08-18 23:04:39,297] Trial 1 finished with value: 0.92515962298571 and parameters: {'n_estimators': 363, 'max_depth': 1, 'learning_rate': 0.6923884519284504, 'gamma': 2.3719556857557514e-07, 'min_child_weight': 9, 'subsample': 0.5195696271131344, 'colsample_bytree': 0.9721890311504119, 'reg_alpha': 0.0004982860931446735, 'reg_lambda': 4.9598622967800296e-08}. Best is trial 0 with value: 0.9636743691091517.
[I 2025-08-18 23:04:39,907] Trial 2 finished with va

Best trial:
F1 Score: 0.977579
Parameters:
n_estimators: 636
max_depth: 3
learning_rate: 0.28911459468011935
gamma: 0.0026877179567177173
min_child_weight: 1
subsample: 0.9884898945689472
colsample_bytree: 0.5529899112143574
reg_alpha: 8.780578569353003e-06
reg_lambda: 0.024175358439225716


**LightGBM results:**

In [17]:
import lightgbm as lgb


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": trial.suggest_categorical(
            "boosting_type", ["gbdt", "dart", "goss"]
        ),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 20, 1000),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 1e-8, 1.0, log=True),
        "cat_smooth": trial.suggest_int("cat_smooth", 1, 100),
        "cat_l2": trial.suggest_float("cat_l2", 1e-8, 10.0, log=True),
        "verbosity": -1,
    }

    model = lgb.LGBMClassifier(**params)

    scores = cross_val_score(
        estimator=model,
        X=X,
        y=y,
        scoring="f1_weighted",
        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
        n_jobs=-1,
    )

    return scores.mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

best_trial = study.best_trial

print("Best trial:")
print(f"F1 Score: {best_trial.value:.6f}")
print("Parameters:")
for k, v in best_trial.params.items():
    print(f"{k}: {v}")

[I 2025-08-18 21:38:31,112] A new study created in memory with name: no-name-a2c71fbc-43fe-4d55-a82d-10f7362f12f0
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[LightGBM] [Fatal] Do not support special JSON characters in feature name.
[W 2025-08-18 21:38:31,823] Trial 0 failed with parameters: {'boosting_type': 'gbdt', 'num_leaves': 231, 'learning_rate': 0.004729326807

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classifier/.venv/lib/python3.13/site-packages/lightgbm/sklearn.py", line 1560, in fit
    super().fit(
    ~~~~~~~~~~~^
        X,
        ^^
    ...<12 lines>...
        init_model=init_model,
        ^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/hamidrezakeshavarz/Documents/GitHub/linearboost-classi

**CatBoost results:**

In [10]:
from catboost import CatBoostClassifier


def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 50, 500),
        "depth": trial.suggest_int("depth", 1, 16),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float(
            "bagging_temperature", 0.1, 10.0, log=True
        ),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "grow_policy": trial.suggest_categorical(
            "grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]
        ),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "rsm": trial.suggest_float("rsm", 0.1, 1.0),
        "loss_function": "Logloss",
        "eval_metric": "F1",
        "cat_features": categorical_cols,
        "verbose": 0,
    }

    model = CatBoostClassifier(**params)

    scores = cross_val_score(
        estimator=model,
        X=X,
        y=y,
        scoring="f1_weighted",
        cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
        n_jobs=-1,
    )

    return scores.mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

best_trial = study.best_trial

print("Best trial:")
print(f"F1 Score: {best_trial.value:.6f}")
print("Parameters:")
for k, v in best_trial.params.items():
    print(f"{k}: {v}")

[I 2025-08-17 17:05:16,870] A new study created in memory with name: no-name-58367467-9a1e-4d65-bfaf-46562a4f4de8
[I 2025-08-17 17:05:17,646] Trial 0 finished with value: 0.9336826514116986 and parameters: {'iterations': 447, 'depth': 7, 'learning_rate': 0.010022052741993987, 'l2_leaf_reg': 0.00012108855576002585, 'random_strength': 0.0014307171606371407, 'bagging_temperature': 1.2400472888236214, 'border_count': 118, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 67, 'rsm': 0.7492964415768959}. Best is trial 0 with value: 0.9336826514116986.
[I 2025-08-17 17:05:28,392] Trial 1 finished with value: 0.9357671816047922 and parameters: {'iterations': 320, 'depth': 12, 'learning_rate': 0.005265584834540258, 'l2_leaf_reg': 3.348332251065364e-05, 'random_strength': 0.016130126681163013, 'bagging_temperature': 3.184610728695145, 'border_count': 132, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 16, 'rsm': 0.3661804580910256}. Best is trial 1 with value: 0.9357671816047922.
[I 2025-08-1

Best trial:
F1 Score: 0.948099
Parameters:
iterations: 478
depth: 12
learning_rate: 0.04254101358886379
l2_leaf_reg: 1.7277473564532868e-07
random_strength: 0.0019440779833289569
bagging_temperature: 0.5084286556246295
border_count: 140
grow_policy: Depthwise
min_data_in_leaf: 7
rsm: 0.37971137763889


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "loan_data.csv"

# Load the latest version
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "",
    file_path,
)

df

/var/folders/dj/6m_rn6_56pvb0zb7k0t6bz4r0000gn/T/ipykernel_1127/3819207535.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 25.7k/25.7k [00:00<00:00, 504kB/s]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,LP002953,Male,Yes,3+,Graduate,No,5703,0.0,128.0,360.0,1.0,Urban,Y
377,LP002974,Male,Yes,0,Graduate,No,3232,1950.0,108.0,360.0,1.0,Rural,Y
378,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
379,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y


In [2]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.
